In [93]:
import cv2 as cv
import numpy as np
import shutil
from sklearn.cluster import KMeans
import random
import os

indstillinger

In [94]:
full_set_plader = "Data/KD hele plader"
train_set_plader = "Data/KD train plader"
test_set_plader = "Data/KD Test plader"
train_set_tiles = "Data/KD train tiles" 

train_set_tiles_mixed = train_set_tiles + "/blandet"

template_folder = "Templates"

test_size=0.2

crop_percentage = 90
circle_size = 0.5

n_clusters=7

Laver test split af hele plader

In [95]:
def copy_files(files, destination):
    for f in files:
        shutil.copy(os.path.join(full_set_plader, f), os.path.join(destination, f))

In [96]:
# Sikre, at destinationsmapperne eksisterer
os.makedirs(train_set_plader, exist_ok=True)
os.makedirs(train_set_plader, exist_ok=True)

# Hent alle filnavne fra data-mappen
all_files = [f for f in os.listdir(full_set_plader) if os.path.isfile(os.path.join(full_set_plader, f))]

# Bland listen i state 42 og lav et tilfældigt split
random.Random(42).shuffle(all_files)
split_point = int((1 - test_size) * len(all_files))

# Fordel filerne i train og test mapper
train_files = all_files[:split_point]
test_files = all_files[split_point:]

# Kopier filerne
copy_files(train_files, train_set_plader)
copy_files(test_files, test_set_plader)

Laver tile split

In [97]:
def get_tiles(image):
    # laver en tom liste 
    tiles = []
    # kører et for loop hvor elementer vil blive tilføjet til listen, hvor y repræsenterer en række af billedet
    for y in range(5):
        tiles.append([])
    # kører et nested loop, hvor billedet bliver delt op i en tavel med 25 kvadrater af 100,100 px og tilføjer til listen tiles
        for x in range(5):
            tiles[-1].append(image[y*100:(y+1)*100, x*100:(x+1)*100])
    return tiles

In [98]:
os.listdir(train_set_plader)

['63.jpg',
 '62.jpg',
 '74.jpg',
 '61.jpg',
 '59.jpg',
 '71.jpg',
 '65.jpg',
 '.DS_Store',
 '64.jpg',
 '70.jpg',
 '58.jpg',
 '66.jpg',
 '72.jpg',
 '73.jpg',
 '67.jpg',
 '9.jpg',
 '14.jpg',
 '28.jpg',
 '29.jpg',
 '15.jpg',
 '17.jpg',
 '16.jpg',
 '12.jpg',
 '13.jpg',
 '39.jpg',
 '11.jpg',
 '10.jpg',
 '38.jpg',
 '34.jpg',
 '20.jpg',
 '22.jpg',
 '23.jpg',
 '37.jpg',
 '33.jpg',
 '27.jpg',
 '26.jpg',
 '32.jpg',
 '18.jpg',
 '24.jpg',
 '30.jpg',
 '31.jpg',
 '25.jpg',
 '19.jpg',
 '42.jpg',
 '4.jpg',
 '56.jpg',
 '5.jpg',
 '57.jpg',
 '55.jpg',
 '7.jpg',
 '41.jpg',
 '69.jpg',
 '68.jpg',
 '40.jpg',
 '54.jpg',
 '6.jpg',
 '2.jpg',
 '50.jpg',
 '44.jpg',
 '45.jpg',
 '51.jpg',
 '47.jpg',
 '53.jpg',
 '1.jpg',
 '52.jpg']

In [99]:
def find_templates_in_image(tile, template_folder):
    threshold = 0.7  # Tærskelværdi for at acceptere et match
    detections = []  # Gemmer detekterede positioner og størrelser

    # Tjek om inputbilledet er i farve og konverter til gråskala til behandling
    if tile.ndim == 3:
        gray_tile = cv.cvtColor(tile, cv.COLOR_BGR2GRAY)
    else:
        gray_tile = tile  # Antag at billedet allerede er i gråskala

    # Gå igennem hver template-fil i mappen
    for template_name in os.listdir(template_folder):
        template_path = os.path.join(template_folder, template_name)
        template = cv.imread(template_path, cv.IMREAD_GRAYSCALE)
        if template is None:
            continue

        # Anvend template matching
        res = cv.matchTemplate(gray_tile, template, cv.TM_CCOEFF_NORMED)
        loc = np.where(res >= threshold)

        # Gem positionerne for matches
        for pt in zip(*loc[::-1]):  # Bytter x og y rundt
            detections.append((pt[0], pt[1], template.shape[1], template.shape[0]))

    # Fjern overlappende detections
    filtered_detections = non_max_suppression(detections)

    # Dæk de detekterede områder på det originale farvebillede
    if tile.ndim == 2:
        tile = cv.cvtColor(tile, cv.COLOR_GRAY2BGR)  # Konverter gråskala tilbage til BGR hvis nødvendigt

    for (x, y, w, h) in filtered_detections:
        cv.rectangle(tile, (x, y), (x + w, y + h), (0, 0, 0), -1)  # Tegner sort rektangel

    return len(filtered_detections), tile

def non_max_suppression(detections, overlapThresh=0.5):
    if not detections:
        return []

    boxes = np.array(detections)
    pick = []

    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,0] + boxes[:,2]
    y2 = boxes[:,1] + boxes[:,3]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        overlap = (w * h) / area[idxs[:last]]

        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

    return boxes[pick].tolist()


In [100]:
def zoom_tile(tile, crop_percentage=1):
    if crop_percentage < 0 or crop_percentage > 100:
        raise ValueError("crop_percentage must be between 0 and 100")
    
    height, width = tile.shape[:2]
    crop_size = crop_percentage / 100.0
    start_x = int(width * crop_size / 2)
    start_y = int(height * crop_size / 2)
    end_x = width - start_x
    end_y = height - start_y
    cropped_tile = tile[start_y:end_y, start_x:end_x]
    zoomed_tile = cv.resize(cropped_tile, (width, height))
    return zoomed_tile


In [101]:
def remove_circle_from_tile(tile, circle_size=0.5):
    if circle_size <= 0 or circle_size > 1:
        raise ValueError("circle_size must be between 0 and 1")

    height, width = tile.shape[:2]
    center_x, center_y = width // 2, height // 2
    radius = int(min(center_x, center_y) * circle_size)
    mask = np.zeros((height, width), dtype=np.uint8)
    cv.circle(mask, (center_x, center_y), radius, (255, 255, 255), -1)
    circle_removed_tile = cv.bitwise_and(tile, tile, mask=cv.bitwise_not(mask))
    return circle_removed_tile


In [102]:
def process_and_save_tiles(train_set_plader, template_folder, output_folder):
    all_tiles = []
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(train_set_plader):
        if not filename.endswith('.jpg'):
            continue

        image_path = os.path.join(train_set_plader, filename)

        if not os.path.isfile(image_path):
            print(f"Image not found: {image_path}")
            continue

        image = cv.imread(image_path)
        if image is None:
            print(f"Image not found: {image_path}")
            continue

        tiles = get_tiles(image)
        for y, row in enumerate(tiles):
            for x, tile in enumerate(row):
                crowns, tile = find_templates_in_image(tile, template_folder)
                
                tile = zoom_tile(tile)
                tile = remove_circle_from_tile(tile)
                all_tiles.append(tile)

                # Save the processed tile
                save_path = os.path.join(output_folder, f"{filename}_{x}_{y}_{crowns}crowns.jpg")
                cv.imwrite(save_path, tile)

process_and_save_tiles(train_set_plader=train_set_plader, 
                       template_folder=template_folder,
                       output_folder='tileswithno')